In [ ]:
import numpy as np
import datetime as dt 
import csv
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
year = 2025
rides_df = pd.read_csv(f'tbd_stats_{year}.csv')

att = pd.read_csv(f"sat_attendance_{year}.csv")
rider_cols = att.columns[1:]
attendance_df = att.melt(
    id_vars=["date"],
    value_vars=rider_cols,
    var_name="slot",
    value_name="rider"
)
attendance_df = attendance_df.dropna(subset=["rider"])
attendance_df = attendance_df[attendance_df["rider"].str.strip() != ""]
attendance_df["rider"] = attendance_df["rider"].str.strip()
attendance_df = attendance_df.drop(columns=["slot"])

In [ ]:
attendance_df

In [ ]:
rides_df

## Attendance Leaderboard

In [143]:
# attendance_counts = attendance_df['rider'].value_counts()
# attendance_regular = attendance_counts[attendance_counts > 4].reset_index()
# attendance_regular.columns = ['rider', 'count']

In [150]:
attendance_counts = (
    attendance_df.groupby("rider")["date"]
    .nunique()                     # or .count() if multiple entries per date aren't possible
    .reset_index(name="count")
)
attendance_regular = attendance_counts[attendance_counts["count"] > 4]
regular_riders = attendance_counts[attendance_counts["count"] > 4]["rider"]


In [152]:
fig = px.bar(
    attendance_regular, 
    y="rider", x="count",
    title="Rider Attendance Leaderboard", 
    text_auto=True)
fig.update_layout(
    title_x=0.5,
    width=600,
    height=800,
    margin=dict(l=5, r=5, t=50, b=5),  # left, right, top, bottom margins
    yaxis=dict(title=None, categoryorder='total ascending'),
    xaxis_title='Number of Rides',
    barmode='stack',
)

fig.show()
fig.write_image(f'{year}/attendance_counts_{year}.png', format='png')

In [147]:
rides_df["date"] = pd.to_datetime(rides_df["date"]).dt.normalize()
attendance_df["date"] = pd.to_datetime(attendance_df["date"]).dt.normalize()

In [164]:

def longest_streak(dates):
    dates = sorted(dates)
    if not dates: return 0

    streak = 1
    max_streak = 1

    for i in range(1, len(dates)):
        # Change 7 days to whatever your normal cadence is
        if (dates[i] - dates[i-1]).days == 7: streak += 1
        else: streak = 1
        max_streak = max(max_streak, streak)
    return max_streak

streak_df = (
    attendance_df.groupby("rider")["date"]
    .apply(lambda d: longest_streak(list(d)))
    .reset_index(name="longest_streak")
)
streak_df = streak_df[streak_df["rider"].isin(regular_riders)]
streak_df.sort_values("longest_streak", ascending=False, inplace=True)


In [ ]:
def longest_hiatus(dates):
    dates = sorted(dates)
    if len(dates) < 2:
        return 0
    gaps = [(dates[i] - dates[i-1]).days for i in range(1, len(dates))]
    return max(gaps)

hiatus_df = (
    attendance_df.groupby("rider")["date"]
    .apply(lambda d: longest_hiatus(list(d)))
    .reset_index(name="longest_hiatus_days")
)
hiatus_df = hiatus_df[hiatus_df["rider"].isin(regular_riders)]
hiatus_df.sort_values("longest_hiatus_days", ascending=False, inplace=True)


,rider,longest_hiatus_days
102,Sten,147
85,Milla,126
50,Hardy,106
34,Constance,105
32,Christian,91
33,Conor,91
37,David M,84
109,Vignesh,84
25,Ben Law,77
68,Katie,70


In [166]:

fig_streak = px.bar(
    streak_df,
    x="rider",
    y="longest_streak",
    title="Longest Attendance Streak per Rider"
)
fig_streak.show()

## Coffee Shops Sunburst

In [ ]:
rides_df

In [ ]:
rides_df['trail_list'] = rides_df['route'].str.split("-")
exploded_rides_df = rides_df.explode('trail_list')


In [ ]:
fig = px.sunburst(exploded_rides_df, path=['trail_list','coffee_shop'])
fig.update_layout(
    width=450,
    height=800,
    margin=dict(l=5, r=5, t=100, b=5),  # left, right, top, bottom margins
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)
# fig.update_traces(textinfo='label')
fig.show()
fig.write_image(f'{year}/coffee_distribution_{year}.png', format='png')

## Ride sizes

In [ ]:
official_rides = rides_df[rides_df["official_ride"] == True]
official_dates = official_rides["date"].unique()
attendance_official = attendance_df[attendance_df["date"].isin(official_dates)]
attendance_counts = attendance_official.groupby("date")["rider"].nunique().reset_index()
attendance_counts.rename(columns={"rider": "attendance"}, inplace=True)
attendance_counts["date"] = pd.to_datetime(attendance_counts["date"]).dt.normalize()
attendance_counts = attendance_counts.sort_values("date")

In [ ]:
attendance_counts

In [ ]:
fig = px.line(
    attendance_counts, x="date", y="attendance", 
    markers=True, 
    title=f"TBD {year} Ride Attendance",
    # line_shape='spline'
)
fig.update_xaxes(
    fixedrange=False,
    dtick="M1",
    tickformat="%b",
    range=[pd.to_datetime(f'{year}-01-01'),pd.to_datetime(f'{year}-12-31')],
    gridcolor='silver'
)
fig.update_layout(
    title_x=0.5,
    width=800,
    height=400,
    margin=dict(l=5, r=5, t=50, b=5),  # left, right, top, bottom margins
    yaxis=dict(title=None),
    yaxis_title='Number of Riders',
    plot_bgcolor='#fffcf7',
    paper_bgcolor='#fffcf7',
    # template='simple_white',
)
fig.update_traces(
    connectgaps=True, 
    line=dict(width=5),
)
fig.show()


In [ ]:

fig.update_layout(
    title_x=0.5,
    width=450,
    height=800,
    margin=dict(l=5, r=5, t=50, b=5),  # left, right, top, bottom margins
    yaxis=dict(title=None),
    yaxis_title='Number of Riders',
    plot_bgcolor='#fffcf7',
    paper_bgcolor='#fffcf7',
    # template='simple_white',
)
fig.update_traces(
    connectgaps=True, 
    line=dict(width=5),
)
fig.show()
fig.write_image(f'{year}/{year}_attendance.png', format='png')

In [ ]:
fig = px.scatter(
    attendance_counts,
    x="date",
    y="attendance",
    trendline="lowess"
)
fig.update_layout(
    title_x=0.5,
    width=800,
    height=400,
    margin=dict(l=5, r=5, t=50, b=5),  # left, right, top, bottom margins
    yaxis=dict(title=None),
    yaxis_title='Number of Riders',
    plot_bgcolor='#fffcf7',
    paper_bgcolor='#fffcf7',
    # template='simple_white',
)
fig.show()